In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

In [2]:
import cv2
import numpy as np
import functions
batchsize = 1

#X_train = bonemarrow_cell()
X_train = functions.get_mnist()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
train_loader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(train_loader)

test,test_label = functions.get_test()
test = test[np.argsort(test_label)]
test_label = test_label[np.argsort(test_label)]

Test_label = torch.LongTensor(test_label.astype('int'))
Test = torch.FloatTensor(test)

test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Test,Test_label), shuffle=False, batch_size=64)

In [3]:
rand = 128
dis = 1

class _netG(nn.Module):
    def __init__(self, isize = 32, nz = 149, nc = 3, ngf = 64, n_extra_layers=0):
        super(_netG, self).__init__()
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        return self.main(input)
    
netG = _netG(nz = rand+dis*10)
print (netG)


""" ==================== DISCRIMINATOR  ======================== """

class _netD(nn.Module):
    def __init__(self, isize = 32, nz = 149, nc = 3, ndf = 64, n_extra_layers=0):
        super(_netD, self).__init__()
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
       # main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
       #                 nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main
        
    def forward(self, input):
        return self.main(input)


netD = _netD(nz = rand+dis*10)
print (netD)

_netG (
  (main): Sequential (
    (initial.138-256.convt): ConvTranspose2d(138, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (initial.256.relu): ReLU (inplace)
    (pyramid.256-128.convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): ReLU (inplace)
    (pyramid.128-64.convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.64.relu): ReLU (inplace)
    (final.64-3.convt): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final.3.tanh): Tanh ()
  )
)
_netD (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bi

In [8]:
class _netD_D(nn.Module):
    def __init__(self):
        super(_netD_D, self).__init__()
        self.conv = nn.Conv2d(256, 1, 4, 1, 0, bias=False)
        
    def forward(self, x):
        x = self.conv(x)
        return x
    
class _netD_Q(nn.Module):
    def __init__(self, nd = 10):
        super(_netD_Q, self).__init__()
        # input is Z, going into a convolution
        self.conv = nn.Conv2d(256, 10, 4, 1, 0, bias=False)
        self.softmax = nn.LogSoftmax()

    def forward(self, x):
        x = self.conv(x)
        x = self.softmax(x)
       # x = x.view(64,10)
        return x

netD_D = _netD_D()
netD_Q = _netD_Q()

In [9]:
netD, netG, netD_D, netD_Q = netD.cuda(), netG.cuda(), netD_D.cuda(), netD_Q.cuda()

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

netG.apply(weights_init)
netD.apply(weights_init)
netD_Q.apply(weights_init)
netD_D.apply(weights_init)

_netD_D (
  (conv): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
)

In [10]:
optimizerD = optim.RMSprop([
                {'params': netD.parameters()},
                {'params': netD_D.parameters()}
            ], 0.00005)

optimizerG = optim.RMSprop(netG.parameters(), lr = 0.00005)
 
optimizerQ = optim.RMSprop([
                {'params': netG.parameters()},            
                {'params': netD.parameters()},
                {'params': netD_Q.parameters()},
            ], 0.00005)

input = torch.FloatTensor(batchsize, 3, 32, 32)
noise = torch.FloatTensor(batchsize, rand+10*dis,1 ,1 )

fixed_noise = torch.FloatTensor(np.random.multinomial(batchsize, 10*[0.1], size=1))
c = torch.randn(batchsize, 10)
z = torch.randn(batchsize, rand)

label = torch.FloatTensor(1)

real_label = 1
fake_label = 0

criterion = nn.BCELoss()
criterion_logli = nn.NLLLoss()
criterion_mse = nn.MSELoss()

criterion, criterion_logli, criterion_mse = criterion.cuda(), criterion_logli.cuda(), criterion_mse.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
z, c = z.cuda(), c.cuda()

In [11]:
def sample_c(batchsize):
    rand_c = np.zeros((batchsize,10),dtype='float32')
    for i in range(0,batchsize):
        rand = np.random.multinomial(1, 10*[0.1], size=1)
        rand_c[i] = rand
    
    label_c = np.argmax(rand_c,axis=1)
    label_c = torch.LongTensor(label_c.astype('int'))
    rand_c = torch.from_numpy(rand_c.astype('float32'))
    return rand_c,label_c

def zero_grad():
    netD.zero_grad()
    netD_Q.zero_grad()
    netD_D.zero_grad()
    netG.zero_grad()

def weight_clamp():
    for p in netD.parameters():
            p.data.clamp_(-0.01, 0.01)
    for p in netD_D.parameters():
            p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q.parameters():
            #p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q_2.parameters():
            #p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q_3.parameters():
            #p.data.clamp_(-0.01, 0.01)
        
def generate_fix_noise(dis=1, cont=4, rand=128):
    
    fixed_z = np.random.randn(10,rand).repeat(10,axis=0)
    changing_dis = np.zeros((100,10),dtype = np.float32)
    list = [n for n in range(0,10)]*10
    for i in range(0,100):
        changing_dis[i,list[i]] = 1
    fixed_cont = np.zeros((100,cont),dtype = np.float32)
    map1 = np.concatenate((changing_dis,fixed_cont,fixed_z),axis=1)
    
    lst = [map1.astype(np.float32)]
    single_cont = np.asarray([float(n-5)*2/5 for n in range(0,10)]*10,dtype = np.float32)
    
    fixed_dis = np.zeros((100,10),dtype=np.float32)
    for t in range(0,5):
        fixed_dis[t*20:t*20+20,t*2] = 1
        
    for t in range (0,4):
        fixed_cont = np.zeros((100,cont),dtype = np.float32)
        fixed_cont[:,t] = single_cont
        map2 = np.concatenate((fixed_dis,fixed_cont,fixed_z),axis=1)
        lst.append(map2.astype(np.float32))
    
    return lst

def fix_noise(dis=1, rand=128):
        
    fixed_z = np.random.randn(10,rand).repeat(10,axis=0)
    changing_dis = np.zeros((100,10),dtype = np.float32)
    list = [n for n in range(0,10)]*10
    for i in range(0,100):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(100,138,1,1)


one = torch.FloatTensor([1])
mone = one * -1
one = one.cuda()
mone = mone.cuda()
fixed_noise = torch.from_numpy(fix_noise()).cuda()

In [ ]:
gen_iterations = 0

for epoch in range(100000):

    dataiter = iter(train_loader)
    i = 0
    
    while i < len(train_loader):
        weight_clamp()
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 1
        else:
            Diters = 1
        
        j = 0
        while j < Diters and i < len(train_loader):
            j += 1
            image_, _ = dataiter.next()
            _batchsize = image_.size(0)
            
            image_ = image_.cuda()
            
            i +=1
            weight_clamp()
    #train on D
    #sending real data 
            zero_grad()
            input.resize_as_(image_).copy_(image_)
            inputv = Variable(input)
         #   label.data.resize_(1).fill_(real_label)
            D_real =netD_D(netD(inputv)).mean(0).view(1)
            #D_loss_real = criterion(D_real, label)
            D_real.backward(one)

    #sending noise
            z.normal_(0, 1)
            rand_c,label_c = sample_c(batchsize)
            c.copy_(rand_c)
            noise = torch.cat([c,z],1)
            noise_resize = noise.view(batchsize,rand+10*dis,1,1)
            noisev = Variable(noise_resize)
            
            G_sample = Variable(netG(noisev).data)
            inputv = G_sample
            D_fake = netD_D(netD(inputv)).mean(0).view(1)
         #   label.data.resize_(1).fill_(fake_label)
           # D_loss_fake = criterion(D_fake, label)
            D_fake.backward(mone)
        
    # update D
            optimizerD.step()
    
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        for p in netD_D.parameters():
            p.requires_grad = False # to avoid computation

    # update G  
        zero_grad()
        noisev = Variable(noise_resize)
        G_sample = netG(noisev)
        D_fake = netD_D(netD(G_sample)).mean(0).view(1)
      #  label.data.resize_(1).fill_(real_label)
       # G_loss = criterion(D_fake, label)
        D_fake.backward(one)
        optimizerG.step()
        
        gen_iterations += 1
        
        for p in netD.parameters():
            p.requires_grad = True # to avoid computation
        for p in netD_D.parameters():
            p.requires_grad = True # to avoid computation

    # update Q
        zero_grad()
        noisev = Variable(noise_resize)
        G_sample = netG(noisev)
        Q_c_given_x = netD_Q(netD(G_sample)).view(batchsize, 10)
        
        crossent_loss = criterion_logli(Q_c_given_x ,Variable(label_c.cuda()))
       # print (Q_c_given_x)

        # ent_loss = torch.mean(-torch.sum(c * torch.log(c + 1e-8), dim=1))
       # ent_loss_2 = torch.mean(-torch.sum(c2 * torch.log(c2 + 1e-8), dim=1))
       # ent_loss_3 = torch.mean(-torch.sum(c3 * torch.log(c3 + 1e-8), dim=1))

        mi_loss = 1*crossent_loss

        mi_loss.backward()
        optimizerQ.step()
        
        if gen_iterations % 20 == 0 :
            errD = D_real - D_fake
            #with open("output_cell.txt","w") as f:
                #f.write('{0} {1} {2} {3}'.format(epoch, gen_iterations , -errD.data[0] , mi_loss.data[0]) + '\n')
            print ('{0} {1} {2} {3}'.format(epoch, gen_iterations , -errD.data[0] , mi_loss.data[0]))
            G_sample = netG(Variable(fixed_noise))
            vutils.save_image(G_sample.data, 'fake.png',nrow=10,normalize=True)
            
            #vutils.save_image(G_sample.data, '{0}fake_samples_{1}.png'.format(-errD.data[0], gen_iterations))
            #vutils.save_image(G_sample.data, 'fake_samples.png',normalize = True)
            
            #for t in range(0,5):
                #fixed_noise = generate_fix_noise(dis, cont, rand)[t].reshape(100,rand+dis*10+cont,1,1)
                #G_sample = netG(Variable(torch.FloatTensor(fixed_noise).cuda()))
                #vutils.save_image(G_sample.data, 'map_%d_cell.png'% (t),nrow=10,normalize=True)

            #torch.save(netG.state_dict(), './params/0517/tumor_netG_epoch_%d.pth' % (epoch))
            #torch.save(netD.state_dict(), './params/0517/tumor_netD_epoch_%d.pth' % (epoch))
            #torch.save(netD_D.state_dict(), './params/0517/tumor_netD_D_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q.state_dict(), './params/0517/tumor_netD_Q_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q_2.state_dict(), './params/0517/tumor_netD_Q_2_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q_3.state_dict(), './params/0517/tumor_netD_Q_3_epoch_%d.pth' % (epoch))
        
        #storage = np.zeros((100,3,64,64),dtype=np.float32)
        #z_fix = Variable(torch.randn(1,128,1,1).cuda().normal_(0, 1))

0 20 0.102350078523 2.3560256958
0 40 0.123174235225 2.32201743126
0 60 0.151501715183 2.26881814003
0 80 0.154200568795 2.25768756866
0 100 0.173340946436 2.35595011711
0 120 0.277534276247 2.30115556717
0 140 0.30385017395 2.29631900787
0 160 0.360038459301 2.26608991623
0 180 0.275202691555 2.32828903198
0 200 0.28527033329 2.30534648895
0 220 0.384680211544 2.29113483429
0 240 0.461592048407 2.28099107742
0 260 0.452832490206 2.31161189079
0 280 0.433574140072 2.28909635544
0 300 0.486361265182 2.35333514214
0 320 0.496575176716 2.34899044037
0 340 0.23903362453 2.2552947998
0 360 0.446248531342 2.27157378197
0 380 0.533320367336 2.28452038765
0 400 0.499164581299 2.39379668236
0 420 0.483726590872 2.25278687477
0 440 0.522290229797 2.25184345245
0 460 0.567592740059 2.3316013813
0 480 0.577718198299 2.43462157249
0 500 0.556218504906 2.3503677845
0 520 0.347980111837 2.30041480064
0 540 0.466526806355 2.26800513268
0 560 0.57599824667 2.33138227463
0 580 0.551301240921 2.251873016

0 4620 0.402835309505 2.32261276245
0 4640 0.445133984089 2.15968680382
0 4660 0.212564229965 2.24700045586
0 4680 0.271957486868 2.11799311638
0 4700 0.285151958466 2.26704907417
0 4720 0.267690002918 2.33726215363
0 4740 0.322572350502 2.24061608315
0 4760 0.372621297836 2.07897019386
0 4780 0.196546554565 2.19915437698
0 4800 0.199357613921 2.26744103432
0 4820 0.377522289753 2.28114795685
0 4840 0.448346436024 2.26423287392
0 4860 0.450548529625 2.26780152321
0 4880 0.270888864994 2.00435733795
0 4900 0.294172346592 2.21085214615
0 4920 0.192735671997 2.09037685394
0 4940 0.372682452202 2.28424477577
0 4960 0.311163425446 2.26155972481
0 4980 0.357585191727 2.26700568199
0 5000 0.342391818762 2.22403049469
0 5020 0.311383664608 2.36142015457
0 5040 0.455030560493 2.29603934288
0 5060 0.474159628153 2.31657385826
0 5080 0.33135959506 2.36675167084
0 5100 0.48933672905 2.29266619682
0 5120 0.345932155848 2.1603937149
0 5140 0.346766352654 2.27184200287
0 5160 0.438664764166 2.2035670

0 9200 0.140882253647 1.67691659927
0 9220 0.440055787563 1.463296175
0 9240 0.27945458889 1.55164349079
0 9260 0.352734327316 1.65624809265
0 9280 0.133584931493 1.56709671021
0 9300 0.152683973312 1.48276269436
0 9320 0.283417016268 1.4895567894
0 9340 0.360114753246 1.60226249695
0 9360 0.371593892574 1.61938309669
0 9380 0.119965098798 1.58979582787
0 9400 0.421173214912 1.42370939255
0 9420 0.369005978107 1.34710884094
0 9440 0.378039240837 1.41465616226
0 9460 0.353075474501 1.57502710819
0 9480 0.372022330761 1.54189515114
0 9500 0.358451813459 1.45575737953
0 9520 0.110799498856 1.52670931816
0 9540 0.196410924196 1.74097967148
0 9560 0.214034944773 1.38716626167
0 9580 0.471035033464 1.6077337265
0 9600 0.220879822969 1.64079320431
0 9620 0.339791893959 1.32693374157
0 9640 0.250121712685 1.4680120945
0 9660 0.219973370433 1.59658634663
0 9680 0.219922751188 1.6937046051
0 9700 0.222986280918 1.65539872646
0 9720 0.239129662514 1.57886922359
0 9740 0.278705745935 1.36642372608

0 13660 0.278684139252 0.876282274723
0 13680 0.364834010601 0.966524899006
0 13700 0.285026103258 0.786436200142
0 13720 0.444191217422 0.910803258419
0 13740 0.368296205997 0.819348454475
0 13760 0.290873110294 0.800589680672
0 13780 0.195624247193 0.889083385468
0 13800 0.255714923143 0.938914835453
0 13820 0.356394946575 1.50376176834
0 13840 0.303843647242 0.7639285326
0 13860 0.157199859619 0.685393095016
0 13880 0.364422380924 0.80807775259
0 13900 0.203647851944 1.08797335625
0 13920 0.339104920626 0.844871997833
0 13940 0.271087765694 0.830208957195
0 13960 0.312120735645 0.734360635281
0 13980 0.331126093864 0.708277881145
0 14000 0.124758593738 0.846462011337
0 14020 0.294807821512 1.30978918076
0 14040 0.289715498686 0.684246242046
0 14060 0.307674050331 0.793765306473
0 14080 0.123847573996 0.867320895195
0 14100 0.197462409735 0.810134291649
0 14120 0.211295247078 0.778919696808
0 14140 0.0630419626832 0.952782988548
0 14160 0.158452123404 0.754080176353
0 14180 0.1900807

0 18000 0.23130454123 0.331169784069
0 18020 0.165259957314 0.442254424095
0 18040 0.111554175615 0.328907251358
0 18060 0.150774359703 0.307541131973
0 18080 0.22229744494 0.406339019537
0 18100 0.0360636226833 0.5383939147
0 18120 0.245038568974 0.254475176334
0 18140 0.268712341785 0.290212631226
0 18160 0.182132333517 0.407740414143
0 18180 0.199965894222 0.35738503933
0 18200 0.132130235434 0.467789649963
0 18220 0.0363811552525 0.457666754723
0 18240 0.169068366289 0.378103375435
0 18260 0.201862037182 0.460556983948
0 18280 0.339273005724 0.218193650246
0 18300 0.195694476366 0.522536039352
0 18320 0.221805691719 0.321163594723
0 18340 0.198724016547 0.336084365845
0 18360 0.394485473633 0.450610071421
0 18380 0.315671622753 0.368923604488
0 18400 0.060900170356 0.366351366043
0 18420 0.171346649528 0.384153902531
0 18440 0.21214659512 0.332558751106
0 18460 0.238849639893 0.290496170521
0 18480 0.346573114395 0.295841574669
0 18500 0.187029451132 0.268909811974
0 18520 0.334383

...

6 366720 -0.0674161314964 0.000256061553955
6 366740 0.0317360572517 1.76429748535e-05
6 366760 0.106470063329 0.000567436218262
6 366780 0.226301699877 0.000409126281738
6 366800 0.182703703642 2.38418579102e-05
6 366820 0.191091746092 0.00865602493286
6 366840 0.0511777885258 0.000260710716248
6 366860 0.0825752988458 8.36849212646e-05
6 366880 0.216665565968 3.81469726562e-06
6 366900 0.0550751425326 0.00387859344482
6 366920 0.113044172525 3.24249267578e-05
6 366940 0.115771330893 0.00729084014893
6 366960 -0.00514572300017 0.000596523284912
6 366980 0.101675868034 2.45571136475e-05
6 367000 0.0613625496626 0.000153303146362
6 367020 0.173257499933 0.000200510025024
6 367040 0.0521419197321 3.31401824951e-05
6 367060 0.0987279564142 0.00031852722168
6 367080 -0.0651380121708 0.00119304656982
6 367100 0.152787387371 1.52587890625e-05
6 367120 0.105180837214 1.08480453491e-05
6 367140 0.250995308161 3.09944152832e-05
6 367160 0.0551252029836 0.000617504119873
6 367180 0.140047058463 

6 370600 0.045517064631 0.219989299774
6 370620 0.129330188036 0.000139236450195
6 370640 0.199485763907 3.09944152832e-05
6 370660 0.0207067672163 4.19616699219e-05
6 370680 0.0815862640738 5.03063201904e-05
6 370700 0.0122958365828 0.000239133834839
6 370720 0.101508319378 0.000276565551758
6 370740 0.0502838790417 0.00172162055969
6 370760 0.077384069562 1.27553939819e-05
6 370780 0.186483055353 4.95910644531e-05
6 370800 0.119425877929 0.000119686126709
6 370820 0.0300477072597 2.80141830444e-05
6 370840 0.052820391953 9.82284545898e-05
6 370860 0.0691289752722 0.000114917755127
6 370880 -0.0481132306159 0.000110626220703
6 370900 -0.00477009639144 0.000749588012695
6 370920 0.0722688138485 7.20024108887e-05
6 370940 0.102675668895 0.000222682952881
6 370960 0.210763350129 8.24928283691e-05
6 370980 0.218450963497 0.000728130340576
6 371000 0.0316071473062 2.55107879639e-05
6 371020 0.142575263977 3.21865081787e-05
6 371040 0.252154499292 2.36034393311e-05
6 371060 -0.037647549063 

6 374500 0.0923689603806 0.000419616699219
6 374520 0.0486500002444 1.71661376953e-05
6 374540 0.0991031527519 0.000541210174561
6 374560 0.149966493249 8.01086425781e-05
6 374580 0.119146347046 0.000280380249023
6 374600 0.0863139033318 8.6784362793e-05
6 374620 0.0404665581882 4.62532043457e-05
6 374640 0.183006241918 1.89542770386e-05
6 374660 0.146737515926 5.14984130859e-05
6 374680 0.0692888200283 0.00063943862915
6 374700 0.117102339864 0.000185966491699
6 374720 0.0337747707963 7.82012939453e-05
6 374740 0.0921234488487 0.000263214111328
6 374760 0.28912037611 8.10623168945e-05
6 374780 0.144728153944 0.000651836395264
6 374800 0.244709700346 7.9870223999e-06
6 374820 0.17676641047 1.00135803223e-05
6 374840 0.0685033947229 0.00105047225952
6 374860 0.105828613043 0.000205039978027
6 374880 0.167821958661 0.000155925750732
6 374900 0.0684348419309 0.0586357116699
6 374920 0.0871872901917 0.000593662261963
6 374940 0.130670264363 0.000174999237061
6 374960 -0.0706585645676 3.910

6 378380 -0.0186414271593 0.00243473052979
6 378400 0.0488657988608 0.000608921051025
6 378420 0.0571176111698 0.000706195831299
6 378440 -0.0217525921762 0.000244617462158
6 378460 0.0570861287415 6.67572021484e-06
6 378480 0.0275075882673 4.24385070801e-05
6 378500 0.0991539433599 0.000459432601929
6 378520 0.0732047557831 0.00394344329834
6 378540 0.111008271575 2.26497650146e-06
6 378560 0.0747693181038 9.17911529541e-05
6 378580 0.162082388997 0.000130176544189
6 378600 0.0179577264935 7.34329223633e-05
6 378620 0.0874277800322 5.10215759277e-05
6 378640 0.00280956923962 0.000243663787842
6 378660 0.194489061832 7.5101852417e-06
6 378680 0.0228232629597 0.0118927955627
6 378700 0.204949170351 1.04904174805e-05
6 378720 0.209930241108 0.00236988067627
6 378740 0.0731963962317 0.000200033187866
6 378760 0.137136399746 0.00184535980225
6 378780 0.0783806592226 0.00126266479492
6 378800 0.102623321116 1.49011611938e-05
6 378820 0.1643101722 4.41074371338e-05
6 378840 0.139158323407 4.

6 382260 0.130269482732 9.34600830078e-05
6 382280 0.135665506124 5.57899475098e-05
6 382300 0.0948830619454 0.0673675537109
6 382320 -0.0276878718287 0.000135183334351
6 382340 0.104476988316 0.000588893890381
6 382360 -0.069783821702 0.00599050521851
6 382380 0.0634282007813 1.31130218506e-05
6 382400 0.106870234013 2.02655792236e-06
6 382420 0.0130699537694 0.000153064727783
6 382440 0.0827191621065 5.00679016113e-05
6 382460 0.0585529506207 1.33514404297e-05
6 382480 0.0429800450802 0.000238418579102
6 382500 0.123388618231 1.12056732178e-05
6 382520 0.087787732482 3.24249267578e-05
6 382540 0.101471737027 4.64916229248e-06
6 382560 0.185480833054 1.04904174805e-05
6 382580 0.220637395978 0.000726699829102
6 382600 -0.0200729221106 6.12735748291e-05
6 382620 0.194450497627 0.000778198242188
6 382640 0.138088077307 0.000425338745117
6 382660 0.257509529591 0.00655221939087
6 382680 0.100724458694 6.31809234619e-05
6 382700 0.0184185504913 0.0015606880188
6 382720 0.040066190064 0.01